In [75]:
import tarfile

In [76]:
file_path1 = "/Users/gajaj/Downloads/GRCh38.d1.vd1.fa.tar.gz"
output_dir1 = "/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome"

In [77]:
with tarfile.open(file_path1, "r") as tar:
    tar.extractall(path=output_dir1)
    print("Files extracted successfully")

Files extracted successfully


# Linear regression

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [4]:
from src.dataloader import embed

## Functions for matrix aggregation

### Test embedding matrices (skip)

In [3]:
# Example setup: Generate a mock input matrix
# Rows: [one-hot encoding (4 rows for A, T, G, C), CNV loss, CNV gain, open chromatin]
# Columns: Nucleotide positions (e.g., 6000 columns for 6k bp)
np.random.seed(42)  # For reproducibility
n_positions = 6000
mock_matrix = np.random.randint(0, 2, size=(7, n_positions))

In [31]:
print(mock_matrix.shape) # a random matrix of 0 and 1
print(mock_matrix)

(7, 6000)
[[0 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 ...
 [1 1 0 ... 1 1 0]
 [1 1 0 ... 1 1 1]
 [1 1 0 ... 0 1 0]]


In [63]:
mock_matrix1 = np.zeros((7, 6000))
mock_matrix1[1, :] = np.ones(6000)
mock_matrix1

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Agregation function

In [5]:
def aggregate_matrix(matrix, tss_position=2000):
    """
    Create a dataset that combines codon frequencies and aggregated CNV/open chromatin metrics.

    Args:
        matrix (numpy.ndarray): The embedding matrix.
        tss_position (int): The starting position for codon counting (default is 1).

    Returns:
        numpy.ndarray: A 1-row dataset with 64 columns for codon counts and 3 columns for CNV/open chromatin metrics.
    """
    # Extract one-hot DNA rows (first 4 rows: A, T, G, C)
    nucleotides = ['A', 'T', 'G', 'C']
    one_hot_dna = matrix[:4, :]

    # Decode one-hot to nucleotide sequence
    sequence = ''.join(
        nucleotides[np.argmax(one_hot_dna[:, i])]
        for i in range(one_hot_dna.shape[1])
    )

    # Start codon counting from the TSS
    start = tss_position - 1
    codons = [
        sequence[i:i + 3]
        for i in range(start, len(sequence), 3)
        if i + 3 <= len(sequence)
    ]

    # Generate all 64 possible codons
    possible_codons = [a + b + c for a in nucleotides for b in nucleotides for c in nucleotides]

    # Count codon frequencies
    codon_counts = Counter(codons)

    # Ensure all 64 codons are represented in the output (with count 0 if absent)
    codon_counts_array = np.array([codon_counts.get(codon, 0) for codon in possible_codons])

    # Extract CNV and open chromatin rows (last 3 rows)
    cnv_loss = matrix[4, :] #TODO do we take the average across all embedding or also from TSS??
    cnv_gain = matrix[5, :]
    open_chromatin = matrix[6, :]

    # Compute averages
    cnv_loss_avg = np.mean(cnv_loss)
    cnv_gain_avg = np.mean(cnv_gain)
    open_chromatin_avg = np.mean(open_chromatin)

    # Combine codon frequencies and aggregated metrics into one dataset
    aggregated_values = np.array([cnv_loss_avg, cnv_gain_avg, open_chromatin_avg])
    final_dataset = np.concatenate([codon_counts_array, aggregated_values])

    # Create column headings
    codon_headings = possible_codons
    aggregated_headings = ['cnv_loss_avg', 'cnv_gain_avg', 'open_chromatin_avg']
    column_headings = codon_headings + aggregated_headings

    return final_dataset, column_headings


### Tests for aggregate matrix (skip)

In [ ]:
final_dataset, column_headings = aggregate_matrix(mock_matrix)
print(final_dataset)
print(column_headings)

[361.         186.          65.          24.         134.
  64.          46.          23.          80.          29.
  17.           7.          35.          25.          15.
   3.         158.          66.          42.          14.
  67.          27.          15.          10.          39.
   9.           8.           2.          15.          10.
   7.           2.          73.          31.          20.
  11.          42.          22.          12.           4.
  12.           7.           1.           2.          13.
   3.           4.           0.          45.          22.
  12.           7.          17.           8.           1.
   0.           5.           3.           4.           2.
   4.           3.           5.           0.           0.5015
   0.50916667   0.49933333]
['AAA', 'AAT', 'AAG', 'AAC', 'ATA', 'ATT', 'ATG', 'ATC', 'AGA', 'AGT', 'AGG', 'AGC', 'ACA', 'ACT', 'ACG', 'ACC', 'TAA', 'TAT', 'TAG', 'TAC', 'TTA', 'TTT', 'TTG', 'TTC', 'TGA', 'TGT', 'TGG', 'TGC', 'TCA', 'TCT', 'TC

In [65]:
final_dataset, column_headings = aggregate_matrix(mock_matrix1)
print(final_dataset)

[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0. 2000.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.]


## Load data: gene positions and classification

Import dataset of gene positions (ignore)

In [ ]:
#gene_df = pd.read_csv('/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/gene_positions.csv')
#gene_df = gene_df[['start', 'end', 'gene_id']] #We leave out the chromosome
#gene_df

Import classification dataset

In [6]:
classification_df = pd.read_csv('/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/classification_median.tsv', sep="\t")
classification_df

,barcode,gene_id,expression_count,classification
0,AAACAGCCAAACGCGA-2,ENSG00000235146,0.000000,zero
1,AAACAGCCAAACGCGA-2,ENSG00000187634,0.240150,low
2,AAACAGCCAAACGCGA-2,ENSG00000187583,0.000000,zero
3,AAACAGCCAAACGCGA-2,ENSG00000205231,0.000000,zero
4,AAACAGCCAAACGCGA-2,ENSG00000228750,0.000000,zero
...,...,...,...,...
27353995,TTTGTTGGTTGAGGTC-3,ENSG00000198886,1.907476,high
27353996,TTTGTTGGTTGAGGTC-3,ENSG00000198786,1.573612,high
27353997,TTTGTTGGTTGAGGTC-3,ENSG00000198727,3.001725,high
27353998,TTTGTTGGTTGAGGTC-3,ENSG00000277666,0.000000,zero


Sets of genes and cells that were preprocessed

In [7]:
barcode_set = set(classification_df['barcode'])
gene_set = set(classification_df['gene_id'])

In [39]:
len(barcode_set)

13677

We have to make sure that the barcodes are also in the epiAneufinder file

In [8]:
cnv_path = 'preprocessing/Multiome/epiAneufinder.tsv'
epiAneu_df = pd.read_csv(cnv_path, sep='\t')

In [9]:
column_names_str = str(epiAneu_df.columns.to_list()[0])
column_names_list = column_names_str.split(' ')
column_names_list = column_names_list[4:]

In [11]:
column_names_set = set(column_names_list)
print(column_names_set)

{'cell-TGCATCCTCCCTCAAC-4', 'cell-AGTCTTGCAGGAATCG-1', 'cell-GGCAGGATCCGGCTAA-1', 'cell-CTAGATTCAGCCTAAC-3', 'cell-CGTGTTACACACAATT-3', 'cell-TGTGTTAAGAGAAGGG-1', 'cell-CACTTTGTCTAACTGA-1', 'cell-TGCTTGTGTTGTTCAC-3', 'cell-TTGGATATCGCTAAGT-3', 'cell-GTCTAACAGCGGTTAT-1', 'cell-TGTTAGCAGGAAGCAC-4', 'cell-GCGGAACCAGCAACCT-3', 'cell-CCACAGGGTTTGGTTC-4', 'cell-TTTCGTCCACCATATG-4', 'cell-GCTCAACCACGTAATT-2', 'cell-TGATTCAAGCTTAGCG-2', 'cell-CATTGCGAGCCAGTAT-3', 'cell-AAGGTATAGGAGTCTT-3', 'cell-CCATCACTCACGCGGT-4', 'cell-ACTATCCGTTTAACGG-1', 'cell-CTAGGACGTTAACGGC-2', 'cell-ACACTAATCCTTGCAC-2', 'cell-ACAGGTAAGCATTATG-2', 'cell-GGTCAAGCAATACTGT-1', 'cell-ATGCAAACAACTAACT-4', 'cell-CCTTGCGTCCAATTAG-4', 'cell-ACACTAATCCGTGACA-2', 'cell-GTTCCCAGTACCTTAC-4', 'cell-AGTAATCGTGATGAGG-3', 'cell-CTTCATCCACGGTACT-3', 'cell-ATCCGTGAGCGAGCGA-4', 'cell-AATCCGTAGGCGGGTA-4', 'cell-TACGTTAAGCCTTAAA-4', 'cell-TCCCTGGTCTTTAAGG-4', 'cell-TTGCTCTCAGGACACA-4', 'cell-AAGCAAGTCCAGCACA-4', 'cell-CGTGTGTCAGTTATCG-4', 

In [40]:
# Filter the set to keep only elements also in the list
barcode_set_filtered = barcode_set.intersection(column_names_list)
len(barcode_set_filtered)

0

In [41]:
# Filter the list to keep only elements also in the set
filtered_list = [item for item in column_names_list if item in barcode_set]
len(filtered_list)

0

### Test data (skip)

In [35]:
#Last 100 elements of the dataset
class_df_test = classification_df.iloc[27353895:27353995]
barcode_set_testing = set(class_df_test['barcode'])
genes_set_testing = set(class_df_test['gene_id'])
print(barcode_set_testing)
print(genes_set_testing)

{'TTTGTTGGTTGAGGTC-3'}
{'ENSG00000241743', 'ENSG00000198712', 'ENSG00000226679', 'ENSG00000011201', 'ENSG00000102271', 'ENSG00000278847', 'ENSG00000068366', 'ENSG00000236091', 'ENSG00000078596', 'ENSG00000101846', 'ENSG00000156298', 'ENSG00000100311', 'ENSG00000165583', 'ENSG00000283697', 'ENSG00000234688', 'ENSG00000286326', 'ENSG00000230668', 'ENSG00000147113', 'ENSG00000160219', 'ENSG00000077279', 'ENSG00000225689', 'ENSG00000182162', 'ENSG00000280870', 'ENSG00000012817', 'ENSG00000212907', 'ENSG00000188511', 'ENSG00000260683', 'ENSG00000169306', 'ENSG00000286050', 'ENSG00000198763', 'ENSG00000227042', 'ENSG00000198804', 'ENSG00000013619', 'ENSG00000227610', 'ENSG00000233103', 'ENSG00000100362', 'ENSG00000183878', 'ENSG00000198888', 'ENSG00000147010', 'ENSG00000100336', 'ENSG00000198947', 'ENSG00000089472', 'ENSG00000235111', 'ENSG00000189108', 'ENSG00000099725', 'ENSG00000286381', 'ENSG00000129824', 'ENSG00000176728', 'ENSG00000228253', 'ENSG00000228343', 'ENSG00000231535', 'ENSG00

### Create a merged dataset (skip)

Match the datasets by the gene_ids

In [9]:
# Merge the datasets based on the gene_id column
merged_df = pd.merge(
    classification_df, gene_df,
    on="gene_id",  # Match rows based on this column
    how="inner"    # Keep only rows with matches in both datasets
)

# Display the result
print(merged_df)

                     barcode          gene_id  expression_count  \
0         AAACAGCCAAACGCGA-2  ENSG00000235146          0.000000   
1         AAACAGCCAAACGCGA-2  ENSG00000187634          0.240150   
2         AAACAGCCAAACGCGA-2  ENSG00000187583          0.000000   
3         AAACAGCCAAACGCGA-2  ENSG00000205231          0.000000   
4         AAACAGCCAAACGCGA-2  ENSG00000228750          0.000000   
...                      ...              ...               ...   
26697499  TTTGTTGGTTGAGGTC-3  ENSG00000212907          1.068847   
26697500  TTTGTTGGTTGAGGTC-3  ENSG00000198886          1.907476   
26697501  TTTGTTGGTTGAGGTC-3  ENSG00000198786          1.573612   
26697502  TTTGTTGGTTGAGGTC-3  ENSG00000198727          3.001725   
26697503  TTTGTTGGTTGAGGTC-3  ENSG00000273554          0.000000   

         classification    start      end  
0                  zero   587577   595116  
1                   low   923923   944575  
2                  zero   966482   975865  
3                  

For testing we use first 1000 rows

In [10]:
merged_df_testing = merged_df.head(0)
barcode_set_testing = set(merged_df_testing['barcode'])
genes_set_testing = set(merged_df_testing['gene_id'])
print(barcode_set_testing)

{'AAACAGCCAAACGCGA-2'}


## Import the embedding

We get the embedding matrix as an input with following rows:
- 4 rows for one-hot-encoding
- CNV loss
- CNV gain
-open chromatin

In [19]:
def adjust_barcode_names_for_embedding(barcode_set):
    output_set = set()
    barcode_list = list(barcode_set)
    for b in barcode_list:
        b_string = str(b)
        output_set.add('cell-' + b_string)

    return output_set


In [20]:
adjust_barcode_names_for_embedding(barcode_set_testing)

{'cell-AAACAGCCAGTAAGTA-2'}

In [12]:
#gtf_path = '/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/Homo_sapiens.GRCh38.113.gtf'
fasta_path = '/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/GRCh38.d1.vd1.fa'
atac_path = '/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/overlap_genes_peaks.tsv' 
cnv_path = '/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome/epiAneufinder.tsv'

In [13]:
embedder = embed(
	fasta_path,
	atac_path,
	cnv_path,
	gene_set=gene_set,
	barcode_set=column_names_set, # chnage name of barcodes!
	mode='single_gene_barcode'
)

In [14]:
barcode_id, gene_id, embedding = next(embedder)

In [15]:
print(barcode_id)
print(gene_id)
print(embedding)

cell-GCGCAATGTTGCGGAT-3
ENSG00000187634
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
rowE, _ = aggregate_matrix(embedding)
rowE

array([ 21.,  25.,  28.,  14.,  29.,  37.,  19.,  23.,  42.,  50.,  77.,
        24.,   4.,  16.,  21.,   6.,  26.,  62.,  95.,  18.,  68.,  95.,
        55.,  91.,  24.,  55.,  58.,  17.,  14.,  54.,  97.,  28.,  35.,
        44.,  78.,  14.,  57., 108.,  77.,  65.,  60., 115., 134.,  46.,
         9.,  31.,  77.,  20.,   5.,   2.,  15.,   4.,  28.,  59.,  20.,
        31.,  23.,  39.,  69.,  42.,  11.,  22.,  20.,  14.,   0.,   0.,
         0.])

## Create the covariate matrix and the response vector

We will have a **matrix X** that has per pair (cell_barcode, gene_id) a row of 67 inputs: 64 are the counts of the triplets and 3 are the averages of open cromatine, cnv gain, cnv loss over the nucleotides

There will be 2 **response vectors**:
- y1: has expression counts per pair (cell_barcode, gene_id)
- y2: classification of expression per pair (cell_barcode, gene_id)

Slow function: for 1 barcode and 100 genes it neads 5min

In [19]:
X_output = []
y1 = []
y2 = []

for barcode, gene_id, embed_matrix in embedder:
    print(barcode, gene_id)

    barcode_strip = barcode.strip('cell-')
    
    # Access the row corresponding to (gene_id, barcodes)
    row_class = classification_df.loc[(classification_df["gene_id"] == gene_id) & (classification_df["barcode"] == barcode_strip)]

    #expression = row_class['expression_count'].iloc[0]
    #print(f'Expression: {expression}')
    #classification = row_class['classification'].iloc[0]
    #print(f'Class: {classification}')

    X_row, _ = aggregate_matrix(embed_matrix)
    print(X_row)

    X_output.append(X_row)
    #y1.append(expression)
    #y2.append(classification)


cell-TCCTTAGTCGGGACTC-4 ENSG00000187634
[ 21.  25.  28.  14.  29.  37.  19.  23.  42.  50.  77.  24.   4.  16.
  21.   6.  26.  62.  95.  18.  68.  95.  55.  91.  24.  55.  58.  17.
  14.  54.  97.  28.  35.  44.  78.  14.  57. 108.  77.  65.  60. 115.
 134.  46.   9.  31.  77.  20.   5.   2.  15.   4.  28.  59.  20.  31.
  23.  39.  69.  42.  11.  22.  20.  14.   0.   0.   0.]
cell-ACTATCCGTCTAACCT-1 ENSG00000187634
[ 21.  25.  28.  14.  29.  37.  19.  23.  42.  50.  77.  24.   4.  16.
  21.   6.  26.  62.  95.  18.  68.  95.  55.  91.  24.  55.  58.  17.
  14.  54.  97.  28.  35.  44.  78.  14.  57. 108.  77.  65.  60. 115.
 134.  46.   9.  31.  77.  20.   5.   2.  15.   4.  28.  59.  20.  31.
  23.  39.  69.  42.  11.  22.  20.  14.   0.   0.   0.]
cell-GTGCACGGTCACAAAT-3 ENSG00000187634
[ 21.  25.  28.  14.  29.  37.  19.  23.  42.  50.  77.  24.   4.  16.
  21.   6.  26.  62.  95.  18.  68.  95.  55.  91.  24.  55.  58.  17.
  14.  54.  97.  28.  35.  44.  78.  14.  57. 108.  77. 

KeyboardInterrupt: 

In [21]:
np.array(X_output)

array([[21., 25., 28., ...,  0.,  0.,  0.],
       [21., 25., 28., ...,  0.,  0.,  0.],
       [21., 25., 28., ...,  0.,  0.,  0.],
       ...,
       [21., 25., 28., ...,  0.,  0.,  1.],
       [21., 25., 28., ...,  0.,  0.,  0.],
       [21., 25., 28., ...,  0.,  0.,  0.]])

In [48]:
def create_matrix_X_from_embedding(embedder):
    X_output = []
    y1 = []
    y2 = []

    for barcode, gene_id, embed_matrix in embedder:
        print(barcode, gene_id)

        barcode_strip = barcode.strip('cell-')
        
        # Access the row corresponding to (gene_id, barcodes)
        row_gene = gene_df.loc[(gene_df["gene_id"] == gene_id)]
        row_class = class_df_test.loc[(class_df_test["gene_id"] == gene_id) & (class_df_test["barcode"] == barcode_strip)]

        start = int(row_gene['start'].iloc[0]) 
        expression = row_class['expression_count'].iloc[0]
        classification = row_class['classification'].iloc[0]

        X_row, _ = aggregate_matrix(embed_matrix, start)
        print(X_row)

        X_output.append(X_row)
        y1.append(expression)
        y2.append(classification)

    return np.array(X_output), np.array(y1), np.array(y2)


    

In [49]:
X_out, y1, y2 = create_matrix_X_from_embedding(embedder)

cell-TTTGTTGGTTGAGGTC-3 ENSG00000182162
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cell-TTTGTTGGTTGAGGTC-3 ENSG00000182162
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cell-TTTGTTGGTTGAGGTC-3 ENSG00000182162
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cell-TTTGTTGGTTGAGGTC-3 ENSG00000182162
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cell-TTTGTTGGTTGAGGT

RuntimeError: generator raised StopIteration

## Linear regression 1

Fit a linear regression model with covariate matrix X and response vector y1

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [85]:
# Step 1: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y1_final, test_size=0.2, random_state=42)

In [86]:
# Step 2: Initialize and fit a Ridge regression model
ridge_model = Ridge(alpha=1.0)  # Regularization strength (alpha=0 means no regularization)
ridge_model.fit(X_train, y_train)

Ridge()

In [87]:
# Step 3: Make predictions
y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)

In [88]:
# Step 4: Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

In [89]:
print(train_mse)
print(test_mse)

0.020568306251147205
0.018756714970882847


: 

**QUESTIONS**:
- The gene expression can be different for same gene in different cells? It does not just refer to a gene_id?
- How do we know what is the end site for the embedding count, is it the gene_end?
- Is in the embedding the chromosome also important?